In [72]:
import pandas as pd

In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
adf=pd.read_csv('/content/drive/MyDrive/Bluetick.ai/users_activity_log.csv')



In [75]:
adf

,user_id,occurred_at,event_name,os,os_version,device_id
0,4436047,2022-10-04 15:18:11,watchlists,android,12,146
1,905476,2022-08-31 15:19:05,home,windows,10,511
2,5038318,2022-10-10 08:33:38,watchlists,windows,10,512
3,489996,2022-09-09 08:54:41,library,windows,10,511
4,3065600,2022-09-03 17:39:05,settings,windows,10,511
...,...,...,...,...,...,...
368709,5651753,2022-09-27 12:34:21,home,windows,10,512
368710,7962325,2022-09-14 13:46:33,run_query,android,12,220
368711,6411131,2022-09-03 10:40:13,watchlists,ios,12,508
368712,4748781,2022-09-19 04:35:20,library,android,11,400


# All Insights are given Considering a sequence of activities done by a user without a 10-minute break between any two activities as one session. If there's a gap of more than 10 minutes between a new activity and previous activity, consider the new activity to be in a new session.

In [76]:


# Converting occurred_at column to datetime
adf['occurred_at'] = pd.to_datetime(adf['occurred_at'], format="%Y-%m-%d %H:%M:%S")

# Sorting the DataFrame by user_id and occurred_at
df = adf.sort_values(['user_id', 'occurred_at'])

# Initializing variables for session tracking
session_start_time = df['occurred_at'].iloc[0]
session_user_id = df['user_id'].iloc[0]

# List to store the sessions
sessions = []

# Iterate through the DataFrame to create sessions which exceds the time limit of 10 minute is cosidered to be a new session
for index, row in df.iterrows():
    if row['user_id'] == session_user_id and (row['occurred_at'] - session_start_time).seconds <= 600:
        continue  # Extend the current session
    else:
        sessions.append((session_user_id, session_start_time, row['occurred_at'],row['event_name'],row['os_version'],row['os'],row['device_id']))
        session_user_id = row['user_id']
        session_start_time = row['occurred_at']

# Append the last session
sessions.append((session_user_id, session_start_time, df['occurred_at'].iloc[-1],df['event_name'].iloc[-1],df['os_version'].iloc[-1],df['os'].iloc[-1],df['device_id'].iloc[-1]))

# Create a new DataFrame with sessions
session_df = pd.DataFrame(sessions, columns=['user_id', 'session_start', 'session_end','event_name','os_version','os','device_id'])

# Display the resulting sessions
session_df


,user_id,session_start,session_end,event_name,os_version,os,device_id
0,9750,2022-08-28 19:16:39,2022-08-30 10:54:50,login,11,android,380
1,9750,2022-08-30 10:54:50,2022-09-05 08:50:01,login,11,android,380
2,9750,2022-09-05 08:50:01,2022-09-05 14:12:02,login,11,android,380
3,9750,2022-09-05 14:12:02,2022-09-08 15:01:53,login,11,android,380
4,9750,2022-09-08 15:01:53,2022-09-10 08:50:15,login,11,android,380
...,...,...,...,...,...,...,...
39970,10013277,2022-10-07 13:39:47,2022-09-04 07:56:44,login,14,ios,503
39971,10013985,2022-09-04 07:56:44,2022-09-06 17:55:09,login,14,ios,503
39972,10013985,2022-09-06 17:55:09,2022-09-12 16:38:45,login,10,windows,512
39973,10013985,2022-09-12 16:38:45,2022-09-25 07:55:29,login,14,ios,503


In [77]:
#Counting the number of session of search happened
no_session_df=session_df.groupby('event_name').count()
no_session_df

,user_id,session_start,session_end,os_version,os,device_id
event_name,,,,,,
autocomplete_engine,5,5,5,5,5,5
home,164,164,164,164,164,164
library,106,106,106,106,106,106
login,35836,35836,35836,35836,35836,35836
query_result_0,239,239,239,239,239,239
query_result_1,2,2,2,2,2,2
query_result_10,2,2,2,2,2,2
query_result_2,3,3,3,3,3,3
query_result_3,4,4,4,4,4,4


In [78]:
#As run_query means the search button is clicked means the count of run_query will be the number of sessions involving the search operation and the autocomplete_engine is also a search events
#So the percentage of session is searched used is
perc=((39975-(47+102+35836+106+164))/39975)*100
print(perc)

9.305816135084429


In [79]:
#Total no of searches:=3720
#Total no of Non successfull searches are the count of query_result_0
#percentage of successfull searches are

print(((3720-239)/3720)*100)

#93% peoples are able to find the search for they are searching

93.5752688172043


In [80]:
# Define the input data as a dictionary
query_results = {
    'query_result_0': 239,
    'query_result_1': 2,
    'query_result_2': 3,
    'query_result_3': 4,
    'query_result_4': 6,
    'query_result_5': 5,
    'query_result_6': 5,
    'query_result_7': 4,
    'query_result_8': 2,
    'query_result_9': 1,
    'query_result_10': 2
}

# Sort the dictionary by values in descending order
sorted_results = dict(sorted(query_results.items(), key=lambda item: item[1], reverse=True))

# Get the top 5 values
top_5 = list(sorted_results.items())[:5]

# Print the top 5 values
for key, value in top_5:
    print(f'{key}: {value}')


query_result_0: 239
query_result_4: 6
query_result_5: 5
query_result_6: 5
query_result_3: 4


As we can say that peoples have to scroll down a bit to get the desired result as the query_result_4,query_result_5,query_result_6,query_result_3 are the most selected search results among the all sesions. So the algorithm need to be refined for better searching experience.

In [81]:
no_session_df.loc['query_result_0']


user_id          239
session_start    239
session_end      239
os_version       239
os               239
device_id        239
Name: query_result_0, dtype: int64

According the output we can see 239 times the users have to search again as the did not get the desired result and gets the error message  which means that the search button is not working that well.

In [82]:
#set display option to show maximum number of rows
grouped_data = session_df.groupby(['event_name', 'os_version']).count()
pd.set_option('display.max_rows', None)
(grouped_data)

user_id  session_start  session_end     os  \
event_name          os_version                                               
autocomplete_engine 10                4              4            4      4   
                    12                1              1            1      1   
home                8                 1              1            1      1   
                    9                 4              4            4      4   
                    10               53             53           53     53   
                    11               11             11           11     11   
                    12               63             63           63     63   
                    13               10             10           10     10   
                    14                8              8            8      8   
                    15               14             14           14     14   
library             8                 1              1            1      1   
                    9                 1              1            1      1   
                    10               34             34           34     34   
                    11                7              7            7      7   
                    12               46             46           46     46   
                    13                8              8            8      8   
                    14                3              3            3      3   
                    15                6              6            6      6   
login               7                43             43           43     43   
                    8               349            349          349    349   
                    9               741            741          741    741   
                    10            11205          11205        11205  11205   
                    11             3046           3046         3046   3046   
                    12            14780          14780        14780  14780   
                    13             1623           1623         1623   1623   
                    14             2074           2074         2074   2074   
                    15             1975           1975         1975   1975   
query_result_0      12              239            239          239    239   
query_result_1      10                1              1            1      1   
                    12                1              1            1      1   
query_result_10     10                1              1            1      1   
                    12                1              1            1      1   
query_result_2      10                1              1            1      1   
                    12                2              2            2      2   
query_result_3      12                3              3            3      3   
                    14                1              1            1      1   
query_result_4      12                4              4            4      4   
                    15                2              2            2      2   
query_result_5      10                2              2            2      2   
                    11                1              1            1      1   
                    12                2              2            2      2   
query_result_6      10                1              1            1      1   
                    12                3              3            3      3   
                    14                1              1            1      1   
query_result_7      10                1              1            1      1   
                    12                1              1            1      1   
                    13                1              1            1      1   
                    15                1              1            1      1   
query_result_8      12                2              2            2      2   
query_result_9      15                1              1

From above result we can see that the OS version 12 is showing error message after searching anything.

In [96]:
#Reset the display option to default

session_df[['os_version','os','device_id']]


,os_version,os,device_id
0,11,android,380
1,11,android,380
2,11,android,380
3,11,android,380
4,11,android,380
...,...,...,...
39970,14,ios,503
39971,14,ios,503
39972,10,windows,512
39973,14,ios,503


In [101]:
#Finding rows which have os version as 12 and event name as query_result_0
version_12 = session_df[ (session_df['os_version'] == 12)&(session_df['event_name']=='query_result_0')]
pd.set_option('display.max_rows', None)


In [102]:
version_12

,user_id,session_start,session_end,event_name,os_version,os,device_id
29,21504,2022-09-13 12:39:19,2022-09-13 13:52:37,query_result_0,12,android,456
38,23857,2022-10-15 14:53:43,2022-10-15 16:05:18,query_result_0,12,android,335
194,83466,2022-09-24 11:37:50,2022-09-26 07:53:20,query_result_0,12,android,87
434,157613,2022-09-11 06:50:00,2022-09-11 08:04:19,query_result_0,12,android,276
560,189005,2022-09-10 16:53:47,2022-09-10 18:08:19,query_result_0,12,android,241
561,189005,2022-09-10 18:08:19,2022-09-12 15:44:40,query_result_0,12,android,241
683,205925,2022-09-06 14:02:59,2022-09-06 15:15:41,query_result_0,12,android,456
704,212426,2022-09-30 11:45:09,2022-08-28 13:14:47,query_result_0,12,android,63
831,256748,2022-09-12 00:52:07,2022-08-27 07:21:19,query_result_0,12,android,276
906,268776,2022-09-16 21:57:51,2022-09-18 23:11:50,query_result_0,12,android,78


In [86]:
bdf=pd.read_csv('/content/drive/MyDrive/Bluetick.ai/device_details.csv')

In [87]:
bdf

,device_id,name,screen_size,os_version,ram,battery,chipset
0,408,Xiaomi Mi 10T Lite 5G,"6.67""",Android 10,6 GB RAM,4820 mAh Li-Po,Qualcomm SM7225 Snapdragon 750G 5G (8 nm)
1,229,Oppo Find X2 Neo,"6.5""",Android 10,12 GB RAM,4025 mAh Li-Po,Qualcomm SM7250 Snapdragon 765G 5G (7 nm)
2,452,Infinix Zero 5,"5.98""",Android 7.0,6 GB RAM,4350 mAh Li-Ion,Qualcomm SDM730 Snapdragon 730 (8 nm)
3,371,Xiaomi Black Shark 4S,"6.67""",Android 11,8/12 GB RAM,4500 mAh Li-Po,Qualcomm SM8250-AC Snapdragon 870 5G (7 nm)
4,59,Infinix Zero 5G,"6.78""",Android 11,8 GB RAM,5000 mAh Li-Po,Qualcomm SM8150 Snapdragon 855+ (7 nm)
...,...,...,...,...,...,...,...
448,44,Asus ROG Phone 5s Pro,"6.78""",Android 11,18 GB RAM,6000 mAh Li-Po,Qualcomm SM8350 Snapdragon 888+ 5G (5 nm)
449,323,vivo Y33,"6.58""",Android 11,4 GB RAM,5000 mAh Li-Po,Mediatek MT6769V/CU Helio G80 (12 nm)
450,399,Samsung Galaxy A20e,"5.8""",Android 9.0,3 GB RAM,3000 mAh Li-Po,Exynos 7884 (14 nm)
451,277,Infinix Note 11s,"6.95""",Android 11,4/6 GB RAM,5000 mAh Li-Po,Qualcomm SM8150 Snapdragon 855 (7 nm)


In [88]:
#Joining the device details dataset with android_versinon_12 dataset using inner join
join_df=pd.merge(version_12, bdf, on='device_id', how='inner')
join_df


,user_id,session_start,session_end,event_name,os_version_x,os,device_id,name,screen_size,os_version_y,ram,battery,chipset
0,21504,2022-09-13 12:39:19,2022-09-13 13:52:37,query_result_0,12,android,456,Xiaomi Poco M4 5G (India),"6.58""",Android 12,4/6 GB RAM,5000 mAh Li-Po,Mediatek MT6833 Dimensity 700 (7 nm)
1,205925,2022-09-06 14:02:59,2022-09-06 15:15:41,query_result_0,12,android,456,Xiaomi Poco M4 5G (India),"6.58""",Android 12,4/6 GB RAM,5000 mAh Li-Po,Mediatek MT6833 Dimensity 700 (7 nm)
2,358952,2022-09-16 14:03:53,2022-09-03 10:25:25,query_result_0,12,android,456,Xiaomi Poco M4 5G (India),"6.58""",Android 12,4/6 GB RAM,5000 mAh Li-Po,Mediatek MT6833 Dimensity 700 (7 nm)
3,6804569,2022-09-01 08:50:08,2022-09-01 10:03:09,query_result_0,12,android,456,Xiaomi Poco M4 5G (India),"6.58""",Android 12,4/6 GB RAM,5000 mAh Li-Po,Mediatek MT6833 Dimensity 700 (7 nm)
4,6804569,2022-09-20 07:10:21,2022-09-20 08:24:18,query_result_0,12,android,456,Xiaomi Poco M4 5G (India),"6.58""",Android 12,4/6 GB RAM,5000 mAh Li-Po,Mediatek MT6833 Dimensity 700 (7 nm)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,8084792,2022-09-13 06:42:36,2022-09-13 08:00:47,query_result_0,12,android,82,Infinix Smart 6 Plus,"6.82""",Android 12,2/3 GB RAM,5000 mAh Li-Po,Mediatek MT6893 Dimensity 1200 (6 nm)
235,8786559,2022-09-18 08:49:43,2022-09-18 10:03:59,query_result_0,12,android,82,Infinix Smart 6 Plus,"6.82""",Android 12,2/3 GB RAM,5000 mAh Li-Po,Mediatek MT6893 Dimensity 1200 (6 nm)
236,9951086,2022-09-03 18:50:22,2022-09-05 13:36:54,query_result_0,12,android,82,Infinix Smart 6 Plus,"6.82""",Android 12,2/3 GB RAM,5000 mAh Li-Po,Mediatek MT6893 Dimensity 1200 (6 nm)
237,9963639,2022-09-18 19:42:22,2022-09-18 20:56:13,query_result_0,12,android,82,Infinix Smart 6 Plus,"6.82""",Android 12,2/3 GB RAM,5000 mAh Li-Po,Mediatek MT6893 Dimensity 1200 (6 nm)


In [89]:
#Counting the number occurence of the device
device_name=join_df.groupby('name').count()
device_name

,user_id,session_start,session_end,event_name,os_version_x,os,device_id,screen_size,os_version_y,ram,battery,chipset
name,,,,,,,,,,,,
Google Pixel 3a XL,5,5,5,5,5,5,5,5,5,5,5,5
Infinix Smart 6 Plus,7,7,7,7,7,7,7,7,7,7,7,7
OnePlus Ace,6,6,6,6,6,6,6,6,6,6,6,6
OnePlus Ace Racing,11,11,11,11,11,11,11,11,11,11,11,11
OnePlus Nord 2T,9,9,9,9,9,9,9,9,9,9,9,9
Oppo A73 5G,13,13,13,13,13,13,13,13,13,13,13,13
Oppo A77,4,4,4,4,4,4,4,4,4,4,4,4
Oppo F19 Pro+ 5G,12,12,12,12,12,12,12,12,12,12,12,12
Oppo K10 5G,15,15,15,15,15,15,15,15,15,15,15,15


In [90]:
#Rearranging the order in decending
device_name=device_name.sort_values(by='os', ascending=False)
device_name

,user_id,session_start,session_end,event_name,os_version_x,os,device_id,screen_size,os_version_y,ram,battery,chipset
name,,,,,,,,,,,,
vivo Y77e,27,27,27,27,27,27,27,27,27,27,27,27
Samsung Galaxy M13 5G,17,17,17,17,17,17,17,17,17,17,17,17
Oppo Reno8,16,16,16,16,16,16,16,16,16,16,16,16
Oppo K10 5G,15,15,15,15,15,15,15,15,15,15,15,15
vivo Y77 (China),13,13,13,13,13,13,13,13,13,13,13,13
Oppo A73 5G,13,13,13,13,13,13,13,13,13,13,13,13
Oppo F19 Pro+ 5G,12,12,12,12,12,12,12,12,12,12,12,12
Xiaomi 11i HyperCharge 5G,11,11,11,11,11,11,11,11,11,11,11,11
OnePlus Ace Racing,11,11,11,11,11,11,11,11,11,11,11,11


In [91]:
#top 5 devices showing fault in search button
device_name.head()

,user_id,session_start,session_end,event_name,os_version_x,os,device_id,screen_size,os_version_y,ram,battery,chipset
name,,,,,,,,,,,,
vivo Y77e,27,27,27,27,27,27,27,27,27,27,27,27
Samsung Galaxy M13 5G,17,17,17,17,17,17,17,17,17,17,17,17
Oppo Reno8,16,16,16,16,16,16,16,16,16,16,16,16
Oppo K10 5G,15,15,15,15,15,15,15,15,15,15,15,15
vivo Y77 (China),13,13,13,13,13,13,13,13,13,13,13,13


In [92]:
#Which chipsets are mainly showing the error most
dev=join_df.groupby('chipset').count()
dev=dev.sort_values(by='name', ascending=False)
dev.head()

,user_id,session_start,session_end,event_name,os_version_x,os,device_id,name,screen_size,os_version_y,ram,battery
chipset,,,,,,,,,,,,
Mediatek MT6833P Dimensity 810 (6 nm),66,66,66,66,66,66,66,66,66,66,66,66
Mediatek Dimensity 1300 (6 nm),31,31,31,31,31,31,31,31,31,31,31,31
Mediatek MT6833 Dimensity 700 (7 nm),30,30,30,30,30,30,30,30,30,30,30,30
Mediatek Dimensity 920 (6 nm),18,18,18,18,18,18,18,18,18,18,18,18
Mediatek MT6877 Dimensity 900 (6 nm),18,18,18,18,18,18,18,18,18,18,18,18
